In [1]:
# Append rolling_ta to sys path.
import sys
import os

# Assuming rolling_ta is in the same level as tests/
module_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add the path to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from ta.trend import SMAIndicator
from rolling_ta.trend import NumbaSMA
from rolling_ta.data import CSVLoader
from rolling_ta.logging import logger
from time import time

In [3]:
csv_loader = CSVLoader()
data = csv_loader.read_resource()

DEBUG - CSVLoader: Loading from resources/btc_ohlcv.csv


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  10000 non-null  float64
 1   open       10000 non-null  float64
 2   high       10000 non-null  float64
 3   low        10000 non-null  float64
 4   close      10000 non-null  float64
 5   volume     10000 non-null  float64
dtypes: float64(6)
memory usage: 468.9 KB


In [5]:
# Compile njit functions
sma = NumbaSMA(data)
sma.update(data.iloc[0])

In [6]:
%%timeit
for _ in range(10000):
    sma = SMAIndicator(data["close"], 14)
    sma.sma_indicator()

2.49 s ± 54.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
for _ in range(10000):
    NumbaSMA(data, memory=False)

417 ms ± 7.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
slice_a = data.copy().iloc[:100]
slice_b = data.copy().iloc[100:]

In [9]:
start = time()
for _ in range(5):
    rolling = NumbaSMA(slice_a)
    for _, series in slice_b.iterrows():
        rolling.update(series)
logger.info("TIMING: [{:,.20f}s]".format(time() - start))

INFO - TIMING: [1.71388888359069824219s]
